<a href="https://colab.research.google.com/github/ducanh2505/scrape_data/blob/main/utilize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install packages

In [4]:
!pip install requests-html

     |████████████████████████████████| 133kB 4.8MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.26.2
    Uninstalling urllib3-1.26.2:
      Successfully uninstalled urllib3-1.26.2


### import packages and define some variables

In [183]:

from requests_html import HTMLSession
import requests
import json
import re
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [6]:
session = HTMLSession()
home = 'https://gearvn.com'
url = 'https://gearvn.com/collections/laptop'

### def get_from_page(url,sesion)
get all links from page

In [7]:
def get_from_page(url,session=session):  
  r = session.get(url)
  product_row = r.html.find('.product-row')
  return [home + link.find('a[href]',first=True).attrs['href']\
                            for link in product_row]
a = get_from_page(url)

In [8]:
len(a),a

(20,
 ['https://gearvn.com/products/laptop-hp-envy-13-ba1028tu-2k0b2pa',
  'https://gearvn.com/products/laptop-asus-vivobook-d509da-ej116t-new',
  'https://gearvn.com/products/laptop-asus-d409da-ek499t',
  'https://gearvn.com/products/laptop-hp-15s-fq1107tu-193q3pa',
  'https://gearvn.com/products/laptop-acer-aspire-3-a315-56-37dv',
  'https://gearvn.com/products/laptop-asus-x509ja-ej427t',
  'https://gearvn.com/products/laptop-asus-a412fa-ek1187t',
  'https://gearvn.com/products/laptop-asus-experbook-p1410cja-ek354t',
  'https://gearvn.com/products/laptop-acer-aspire-5-a514-54-39ku',
  'https://gearvn.com/products/laptop-asus-vivobook-flip-tm420ia-ec155t',
  'https://gearvn.com/products/laptop-asus-vivobook-a515ea-bq489t',
  'https://gearvn.com/products/laptop-asus-x509ja-ej021t',
  'https://gearvn.com/products/laptop-asus-x509jp-ej012t',
  'https://gearvn.com/products/laptop-acer-aspire-5-a514-53-38gf',
  'https://gearvn.com/products/laptop-asus-vivobook-x409ja-ek014t-silver',
  'htt

### get_from_site()
scrawl all link from the site

In [9]:
def get_from_site(session= session):
  finish = False
  page = 1
  all_links = []
  while not finish:
    t = get_from_page(url+'?page={}'.format(page),session)
    if (len(t) > 0 ):
      page += 1
      all_links.extend(t)
    else: 
      finish = True
  return all_links



In [10]:
all_links =get_from_site()
len(all_links)


102

### get_from_product()
scrawl from a product

In [11]:
product = all_links[0]
product

'https://gearvn.com/products/laptop-hp-envy-13-ba1028tu-2k0b2pa'

In [177]:
r = session.get('https://gearvn.com/products/laptop-gaming-asus-rog-zephyrus-m-gu502gv-az079t')

text = r.html.find('.product_parameters')[0].find('p')[0].text.split("\n")
text = [t.split(':') for t in text]
text


  

[['Nhà sản xuất\xa0', ' Asus'],
 ['Xuất xứ\xa0', ' Chính hãng'],
 ['Bảo hành\xa0', ' 24 Tháng'],
 ['Tình trạng\xa0', ' Mới 100%'],
 ['Màu sắc', '\xa0Black'],
 [''],
 [''],
 [''],
 ['Quà tặng', ''],
 [''],
 ['★\xa0Túi chống shock Gearvn\xa0trị giá\xa0139.000 VNĐ'],
 [''],
 [''],
 [''],
 ['Default Title'],
 ['36,990,000₫'],
 ['- +'],
 ['(Vui lòng chọn số lượng trước khi mua hàng)'],
 ['Giá Cũ', ''],
 ['46,990,000₫'],
 [''],
 ['Giá KM', '36,990,000₫'],
 ['Đặt hàng']]

In [210]:
products = [] 
bar = tqdm(all_links,total = len(all_links))
for product in bar:
  r = session.get(product)
  product_dictionary = {}

  # get general atributes
  general_attributes = r.html.find('table>tbody',first=True).find('tr')
  for attr in general_attributes:   
    if len(attr.find('td')) == 2 :
      product_dictionary[attr.find('td')[0].text] = attr.find('td')[1].text 
                                             


  # get product parameter
  text = r.html.find('.product_parameters')[0].find('p')[0].text.split("\n")
  text = [t.split(':') for t in text]
  for i,t in enumerate(text):
    if t[0] == 'Giá Cũ':
      product_dictionary['Giá Cũ'] = text[i+1][0]      
    elif len(t) == 2:
      product_dictionary[t[0]] = t[1]


  products.append(product_dictionary)

''

In [191]:
all_links[16]

'https://gearvn.com/products/lenovo-thinkbook-14-20rv00b8vn'

In [173]:
import codecs
codecs.encode(text[7][0][0])

b'\xf0\x9f\x8e\x81'

'Chính hãng'